In [1]:
import pandas as pd
import numpy as np
import sys, os, re
import matplotlib.pyplot as plt

# from importlib import reload

if re.match(r'^(\w\:\\)|(/)', os.getcwd()):
    # sys.path.insert(0, "c:/Users/dmitr/devs/quantl/")
    # sys.path.insert(0, "c:/Users/dmitr/devs/Qube/")
    sys.path.insert(0, "c:/Users/dmitr/devs/Qube2exp/src/")
    # sys.path.insert(0, "c:/Users/dmitr/devs/Qube2exp/tests/")
    sys.path.insert(0, "c:/Users/dmitr/devs/Qube2exp/")
    print('installed')


# just some handy helpers from old Qube
import tests.qube.ta.utils_for_testing as test

%load_ext cython

T = pd.date_range('2024-01-01 00:00', freq='1Min', periods=100_000)
ds = 1 + (2*np.random.randn(len(T))).cumsum()
data = list(zip(T, ds))

installed
 > Reloaded c:\Users/dmitr/devs/Qube2exp/src\qube\core\series.pyx


In [7]:
%load_ext autoreload
%autoreload 2

from qube.utils import reload_pyx_module

reload_pyx_module('../../src/qube/core/')
from qube.core.series import TimeSeries
del sys.modules['qube.core.series']
from qube.core.series import (TimeSeries, sma, ema, tema, dema, kama, recognize_time, OHLCV, RollingSum, lag, compare)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
 > Reloaded ../../src/qube/core\series.pyx


# Strategy high level design

In [6]:
class IStrategy:

    def trade(self, position: float, group=None, stop=None, take=None, limit=None, cancel=False):
        print(f"TRADE: {position} [{group}]")
        pass

    def cancel(self, group=None):
        print(f"CANCEL: all from [{group}]")

    def position(self) -> float:
        """
        Returns open position
        """
        return 0.0


class CrossTestClassical(IStrategy):

    def __init__(self) -> None:
        pass

    def on_data(self, xs: OHLCV, is_bar_closed: bool):
        if is_bar_closed:
            fast = ema(xs.close, 15)
            slow = ema(xs.close, 50)
            b = xs[0]

            if (fast[0] > slow[0]) and (fast[1] < slow[1]):
                self.trade(+1, 'long', stop=b.low, cancel=True)

            elif (fast[0] < slow[0]) and (fast[1] > slow[1]):
                self.trade(-1, 'short', stop=b.high, cancel=True)

class CrossTestVector(IStrategy):

    def __init__(self) -> None:
        pass

    def on_data(self, xs: OHLCV, is_bar_closed: bool):
        if is_bar_closed:
            b = xs[0]
            s = compare(ema(xs.close, 15), ema(xs.close, 50))
            where(s, {
                +1: +1,
                -1: +1,
                }
            )

            # if (fast[0] > slow[0]) and (fast[1] < slow[1]):
            #     self.trade(+1, 'long', stop=b.low, cancel=True)

            # elif (fast[0] < slow[0]) and (fast[1] > slow[1]):
            #     self.trade(-1, 'short', stop=b.high, cancel=True)
            

In [15]:
from array import array

In [22]:
a = array('d', [np.nan]*5)

array('d', [nan, nan, nan, nan, nan])

In [28]:
a[4] = 1

In [30]:
a

array('d', [nan, nan, nan, nan, 1.0])

In [11]:
xs = test.push(OHLCV('1h'), data)

In [12]:
s = compare(ema(xs.close, 15), ema(xs.close, 50))

In [ ]:
s.pd()

In [5]:
t = CrossTestClassical()

In [ ]:
fast = ema(xs.close, 15)
slow = ema(xs.close, 50)
where(
    (fast > slow) & (lag(fast, 1) < lag(slow, 1)) & (lag(xs.low, 1)  ), 1
)